# Le but de ce notebook est d'identifier des caractéristiques permettant de différencier des textes écrits par Molière et Corneille

# PRIVE: Hyperparamétres fixés par MathAData

In [ ]:
# nombre de caractéres dans chaque paragraphe dont on veut identifier l'auteur
# 1000 caractères correspond à environ une page de texte (environ 180 mots)
min_paragraph_length = 1000

# on divisie les données d'entraînement en 90% en train et 10% en validation
# une oeuvre d'un auteur donné doit être entièrement soit en train soit en validation
percentage_in_train = 0.9

# on ignore les majuscules / minuscules dans les mots: Monsieur == monsieur
use_lowercase = True

# on ignore les accents:  être == etre
use_diacritics = True

# cette option permet de ne tenir compte que de la racine du mot.
# elle est désactivée car elle dégrade les performances
use_stemming = False

# nombre de mots signatures chez chaque auteur
# on ne regarde que les 'most_common_normalized_words_count' mots les plus courants chez chaque auteur
most_common_normalized_words_count = 50

# Mots vides. 
# Ils sont ignorés par l'outil car très communs à la fois chez Molière et chez Corneille
stop_words = set(["de","et","que","je","a","la","le","ne","ce","il","pour","un","qui","me","est","mais","des","moi","votre","qu'il","lui","du","fait","par","se","au","cette","sur","j'ai","avec","tous","vos","ces","n'est","peu","peut","quelque","dont","quoi","aux","donc","d'une","s'il","notre","sais","donne","vois","m'en","cet","autre","puis","assez","quel","veut","va","ils","doit","ont","vu", "en", "les", "vous", "mon","pas","si","plus", "tout", "nous", "ma", "sans", "ou", "c'est", "bien", "dans","une", "son","tu", "point", "mais", "mes", "d'un", "elle", "ses", "meme", "comme", "te", "sa", "ton", "ta"])


# PRIVE: Méthodes utilisés dans le Notebook

In [ ]:
import functools
from typing import List,Set,Tuple,Dict
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter,MaxNLocator     
import numpy as np
from scipy.interpolate import make_interp_spline
import os
import pathlib
import re
import os
import random
import math
import pandas as pd
from collections import defaultdict

#!pip install unidecode
#!pip install wordcloud
#!pip install pillow


# le répertoire de travail
directory = os.path.abspath('')
random.seed(42)

# retourne tous les fichiers *.txt présents dans le repertoire 'path'
def all_txt_files_in_directory(path: str):
    return [os.path.join(path,f) for f in os.listdir(path) if os.path.isfile(os.path.join(path, f)) and f.endswith('.txt')]

# infique si la ligne 'ligne' est une ligne valide ou si elle doit être ignorée (par exemple si elle vide)
def is_valid_line(line: str) -> bool:
    if line.startswith('Scène ') or line.startswith('Acte '):
        return False
    if len(line)<10:
        return False
    return True

def split_book_into_paragraphs(path: str) -> List[str]:
    result = []
    with open(path, encoding='latin1') as file:
        current_paragraph = ""
        for line in file:
            if is_valid_line(line):
                if current_paragraph :
                    current_paragraph += "\n"
                current_paragraph += line.rstrip()
                if len(current_paragraph) >= min_paragraph_length:
                    result.append(current_paragraph) 
                    current_paragraph = ""
    if len(current_paragraph) >= min_paragraph_length or (current_paragraph and len(result) == 0):
        result.append(current_paragraph) 
    return result

def load_all_books(path: str) -> Dict[str,List[str]]:
    book_to_paragraphs = dict()
    for book_path in all_txt_files_in_directory(path):
        book_to_paragraphs[pathlib.Path(book_path).stem] = split_book_into_paragraphs(book_path)
    return book_to_paragraphs

# pour supprimer les accents
@functools.lru_cache(maxsize=None)
def remove_diacritics(word: str) -> str:
    import unidecode  
    return unidecode.unidecode(word)

# mots en minuscules
@functools.lru_cache(maxsize=None)
def to_lowercase(word: str) -> str:
    return word.lower()

# pour le stemming
from nltk.stem.snowball import FrenchStemmer
stemmer = FrenchStemmer()
@functools.lru_cache(maxsize=None)
def to_stemming(word: str) -> str:
    return stemmer.stem(word)

@functools.lru_cache(maxsize=None)
def compute_normalized_word(word: str) -> str:
    if use_lowercase:
        word = to_lowercase(word)
    if use_diacritics:
        word = remove_diacritics(word)
    if use_stemming:
        word = to_stemming(word)
    return word

def paragraph_count(book_to_paragraphs: Dict[str, List[str]]) -> int:
    if not book_to_paragraphs:
        return 0
    return sum([len(c) for c in book_to_paragraphs.values()])

def split_text(text:str) -> List[str]:
    return re.findall(r"\b[\w'^\d]+\b", text.rstrip())
            
def word_count(book_to_paragraphs: Dict[str, List[str]]) -> int:
    result = 0
    for paragraph in all_paragraphs(book_to_paragraphs):
        result += len(split_text(paragraph.rstrip()))
    return result

def all_paragraphs(book_to_paragraphs: Dict[str, List[str]]) -> List[str]:
    result = []
    for p in book_to_paragraphs.values():
        result.extend(p)
    return result

# reduce the dataset so that it contains exactly 'target_count' paragraphs
def reduce_to_paragraph_count(book_to_paragraphs: dict, target_count: int ) -> int:
    current_count = paragraph_count(book_to_paragraphs)
    if current_count<target_count:
        raise Exception(f'current_count {current_count} < target_count {target_count}')
    to_remove = current_count-  target_count
    result = dict()
    for book, paragraphs in sorted(book_to_paragraphs.items(), key =lambda x : len(x[1])):
        if len(paragraphs)<=to_remove:
            to_remove-=len(paragraphs)
            continue
        result[book] = paragraphs[:len(paragraphs)-to_remove]
        to_remove = 0
    return result


def compute_most_common_normalized_words(normalized_words_to_stats: Dict[str, Tuple[int,Dict[str,int]] ], most_common_count:int) -> Dict[str,float]:
    sorted_by_total_count = sorted(normalized_words_to_stats.items(), key=lambda item:item[1][0], reverse=True)
    total_count = sum([stats[0] for normalized_word,stats in normalized_words_to_stats.items() if normalized_word not in stop_words])
    result  = dict()
    for normalized_word,stats in sorted_by_total_count:
        if normalized_word not in stop_words:
            result[normalized_word] = stats[0]/total_count
        if len(result)>=most_common_count:
            break
    return result

def compute_normalized_words_to_stats(paragraphs: List[str]) -> Dict[str, Tuple[int,Dict[str,int]] ]:
    normalized_words_to_stats = dict()
    for paragraph in paragraphs:
        words = split_text(paragraph)
        for original_word in words:
            normalized_word = compute_normalized_word(original_word)
            if normalized_word not in normalized_words_to_stats:
                normalized_words_to_stats[normalized_word] = (0, dict())
            count,original_word_count = normalized_words_to_stats[normalized_word]
            if original_word not in original_word_count:
                original_word_count[original_word] = 1
            else:
                original_word_count[original_word] += 1
            normalized_words_to_stats[normalized_word] = (count+1,original_word_count)
    return normalized_words_to_stats

def compute_normalized_word_to_original_word(text: str) -> Dict[str,str]:
    normalized_word_to_original_word = dict()
    splitted_text = split_text(text)
    for original_word in splitted_text:
        normalized_word = compute_normalized_word(original_word)
        if normalized_word in stop_words:
            continue
        normalized_word_to_original_word[normalized_word] = original_word
    return normalized_word_to_original_word

def get_max_item(dic: Dict[str, int]) -> Tuple[str,int]:
    key_max_count, max_count = (None, None)
    for key,count in dic.items():
        if max_count is None or count > max_count:
            key_max_count, max_count = key, count
    return key_max_count, max_count
    
def split_train_validation_single_author(book_to_paragraphs: dict, percentage_in_train:float) :
    books = list(book_to_paragraphs.keys())
    train = dict()
    validation = dict()
    for book, paragraphs in book_to_paragraphs.items():
        paragraphs_count = len(paragraphs)
        percentage_in_train_if_adding_to_train = (paragraph_count(train)+len(paragraphs))/max(paragraph_count(train)+paragraph_count(validation)+len(paragraphs),1)
        percentage_in_train_if_adding_to_validation = paragraph_count(train)/max(paragraph_count(train)+paragraph_count(validation)+len(paragraphs),1)
        if abs(percentage_in_train_if_adding_to_train-percentage_in_train)<abs(percentage_in_train_if_adding_to_validation-percentage_in_train):
            train[book] = paragraphs
        else:
            validation[book] = paragraphs
    return train, validation

def split_train_validation_all_authors(book_to_paragraphs_author1: dict, book_to_paragraphs_author2: dict, percentage_in_train:float) :
    train_author1,validation_author1 = split_train_validation_single_author(book_to_paragraphs_author1, percentage_in_train)
    train_author2,validation_author2 = split_train_validation_single_author(book_to_paragraphs_author2, percentage_in_train)

    target_length_validation = min(paragraph_count(validation_author1),paragraph_count(validation_author2))            
    validation_author1 = reduce_to_paragraph_count(validation_author1, target_length_validation)
    validation_author2 = reduce_to_paragraph_count(validation_author2, target_length_validation)

    target_length_train = min(paragraph_count(train_author1),paragraph_count(train_author2))            
    train_author1 = reduce_to_paragraph_count(train_author1, target_length_train)
    train_author2 = reduce_to_paragraph_count(train_author2, target_length_train)

    proportion_in_train = target_length_train/(target_length_train+target_length_validation)
    if proportion_in_train>percentage_in_train:
        target_length_train = int( (percentage_in_train/(1-percentage_in_train)) *target_length_validation )
        train_author1 = reduce_to_paragraph_count(train_author1, target_length_train)
        train_author2 = reduce_to_paragraph_count(train_author2, target_length_train)
    else:
        target_length_validation = int( ((1-percentage_in_train)/percentage_in_train) *target_length_train )
        validation_author1 = reduce_to_paragraph_count(validation_author1, target_length_validation)
        validation_author2 = reduce_to_paragraph_count(validation_author2, target_length_validation)
    return train_author1,validation_author1,train_author2,validation_author2

def compute_error(TP: int, TN: int, FP: int, FN: int):
    return 1-(TP+TN)/max(TP+TN+FP+FN,1)

def calcul_1ere_caracteristique(text:str, most_common_words_for_author: Dict[str,float]) -> float:
    # nombre de mots signatures de l'auteur présents dans le texte 'text'
    valeur_1ere_caracteristique = 0
    for original_word in split_text(text):
        normalized_word = compute_normalized_word(original_word)
        if normalized_word in most_common_words_for_author:
            valeur_1ere_caracteristique += 1
    return valeur_1ere_caracteristique

def compute_confusion_matrix_single_author(author_name:str, texts_from_author: List[str], text_from_other_authors: List[str], most_common_words_from_author: dict , seuil_1ere_caracteristique: int, verbose:bool) ->Tuple[int,int,int,int]:
    TP = 0 # y_true = author_name ,  y_pred = author_name
    TN = 0 # y_true = another author, y_pred = another author
    FN = 0 # y_true = author_name,   y_pred = another author
    FP = 0 # y_true = another_author, y_pred = author_name 
    for t in texts_from_author:
        valeur_1ere_caracteristique = calcul_1ere_caracteristique(t, most_common_words_from_author)
        if valeur_1ere_caracteristique>=seuil_1ere_caracteristique:
            if TP == 0 and verbose:
                print(f'\nExemple de TP (Texte de {author_name}, bien identifié, score {author_name}: {round(valeur_1ere_caracteristique,4)}):\n{t}\n')
            TP += 1
        else:
            if FN == 0 and verbose:
                print(f'\nExemple de FN (Texte de {author_name}, mal identifié, score {author_name}: {round(valeur_1ere_caracteristique,4)}):\n{t}\n')
            FN += 1
    for t in text_from_other_authors:
        valeur_1ere_caracteristique = calcul_1ere_caracteristique(t, most_common_words_from_author)
        if valeur_1ere_caracteristique>=seuil_1ere_caracteristique:
            if FP == 0 and verbose:
                print(f"\nExemple de FP (Texte d'un autre auteur, mal identifié, score {valeur_1ere_caracteristique}: {round(valeur_1ere_caracteristique,4)}):\n{t}\n")
            FP += 1
        else:
            if TN == 0 and verbose:
                print(f"\nExemple de TN (Texte d'un autre auteur, bien identifié, score {valeur_1ere_caracteristique}: {round(valeur_1ere_caracteristique,4)}):\n{t}\n")
            TN += 1
    return (TP,TN,FP,FN)
        
def train_single_author(author_dataset, author_name, other_authors_dataset, seuil_1ere_caracteristique: int, verbose: bool = False) -> Tuple[float, float]:
    random.seed(42)
    if verbose: 
        print(f'\n{author_name} Dataset: {paragraph_count(author_dataset)} paragraphes ({word_count(author_dataset)} mots) venant de {len(author_dataset)} oeuvres:\n{list(author_dataset.keys())}')
        print(f'\nother_authors Dataset: {paragraph_count(other_authors_dataset)} paragraphes ({word_count(other_authors_dataset)} mots) venant de {len(other_authors_dataset)} oeuvres:\n{list(other_authors_dataset.keys())}')

    train_author,validation_author,train_other_authors,validation_other_authors = split_train_validation_all_authors(author_dataset, other_authors_dataset, percentage_in_train)

    if verbose: 
        print(f'\n{author_name} Train Dataset: {paragraph_count(train_author)} paragraphes ({word_count(train_author)} mots) venant de {len(train_author)} oeuvres:\n{list(train_author.keys())}')
        print(f'\n{author_name} Validation Dataset: {paragraph_count(validation_author)} paragraphes ({word_count(validation_author)} mots) venant de {len(validation_author)} oeuvres:\n{list(validation_author.keys())}')
        print(f'\nother_authors Train Dataset: {paragraph_count(train_other_authors)} paragraphes ({word_count(train_other_authors)} mots) venant de {len(train_other_authors)} oeuvres:\n{list(train_other_authors.keys())}')
        print(f'\nother_authors Validation Dataset: {paragraph_count(validation_other_authors)} paragraphes ({word_count(validation_other_authors)} mots) venant de {len(validation_other_authors)} oeuvres:\n{list(validation_other_authors.keys())}')

    train_normalized_words_to_stats_author = compute_normalized_words_to_stats(all_paragraphs(train_author))
    # we only keep the most common words
    train_most_common_author = compute_most_common_normalized_words(train_normalized_words_to_stats_author, most_common_normalized_words_count)
    train_error = compute_error(*compute_confusion_matrix_single_author(author_name,all_paragraphs(train_author), all_paragraphs(train_other_authors), train_most_common_author , seuil_1ere_caracteristique, verbose))
    validation_error = compute_error(*compute_confusion_matrix_single_author(author_name,all_paragraphs(validation_author), all_paragraphs(validation_other_authors), train_most_common_author , seuil_1ere_caracteristique, verbose))
    return train_error,validation_error

def create_table_with_occurences_corneille_moliere(original_words: List[str]) -> pd.DataFrame:

    original_words.sort()
    occurences_moliere = []
    occurences_corneille = []
    frequence_moliere = []
    frequence_corneille = []
    for original_word in original_words:
        normalized_word = compute_normalized_word(original_word)
        occurences_moliere.append(stats_moliere[normalized_word][0] if normalized_word in stats_moliere else 0)
        occurences_corneille.append(stats_corneille[normalized_word][0] if normalized_word in stats_corneille else 0)
        frequence_moliere.append(stats_moliere[normalized_word][0]/moliere_total_word_count_without_stopwords if normalized_word in stats_moliere else 0)
        frequence_corneille.append(stats_corneille[normalized_word][0]/corneille_total_word_count_without_stopwords if normalized_word in stats_corneille else 0)
    df =pd.DataFrame(
        {'Mot': original_words,
        'Occurences Molière': occurences_moliere,
        'Occurences Corneille': occurences_corneille,
        'Fréquence Molière': frequence_moliere,
        'Fréquence Corneille': frequence_corneille} 
        )
    df.set_index(['Mot'],inplace=True)    
    return df   

def display_wordcloud(most_common_normalized_words: dict, normalized_words_to_stats: Dict[str, Tuple[int,Dict[str,int]] ], title:str) -> None:
    from wordcloud import WordCloud
    import matplotlib.pyplot as plt
    word_frequencies = dict()
    for normalized_word, frequency in most_common_normalized_words.items():
        original_word = get_max_item(normalized_words_to_stats[normalized_word][1])[0]
        if normalized_word not in stop_words:
            word_frequencies[original_word] = frequency
    wordcloud = WordCloud(width=1200, height=600, background_color='white').generate_from_frequencies(word_frequencies)
    # Display the generated word cloud
    plt.figure(figsize=(20, 10))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(title, fontsize=20)
    plt.axis('off')
    plt.show()
    
def display_plot(most_common_normalized_words: Dict[str,float], normalized_words_to_stats: Dict[str, Tuple[int,Dict[str,int]] ], author:str, top_k: int, display_in_percentage: bool):
    # Create the bar plot
    keys = []
    values = []
    for normalized_word, frequency in most_common_normalized_words.items():
        stats = normalized_words_to_stats[normalized_word]
        original_word = get_max_item(stats[1])[0]
        if normalized_word not in stop_words:
            keys.append(original_word)
            if display_in_percentage:
                values.append(frequency)
            else:
                values.append(stats[0])
            if len(keys)>=top_k:
                break
    title = f'Les {top_k} mots les plus communs chez {author}'
    display_key_values_plot(keys, values, title, display_in_percentage)

def display_dict_plot(dico, title: str, sort_dictionary:bool, display_in_percentage: bool):
    if sort_dictionary:
        dico = sorted(dico.items(), key = lambda x:x[1], reverse=True)       
    keys = [c[0] for c in dico]
    frequencies = [c[1] for c in dico]
    display_key_values_plot(keys, frequencies, title, display_in_percentage)
    
def display_key_values_plot(keys: List[str], values, title:str, display_in_percentage: bool):
    plt.figure(figsize=(20, 5))
    plt.bar(range(len(keys)), values, color='blue', tick_label=keys)
    # Add title and labels
    plt.title(title, fontsize=25)
    #plt.xlabel('Mot', fontsize=15)
    if display_in_percentage:
        plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
        plt.ylabel(f"Fréquence d'occurences", fontsize=25)
    else:
        plt.gca().yaxis.set_major_locator(MaxNLocator(integer=True))
        plt.ylabel(f"Nombre d'occurences", fontsize=25)
    plt.gca().tick_params(axis='y', which='major', labelsize=15) 
    plt.xticks(rotation=90, fontsize=20)
    plt.xlim(-0.5, len(keys) - 0.5)
    # Display the plot
    plt.show()


## PRIVE: Chargement des données et création d'un fichier de statistiques

In [ ]:
moliere_dataset = load_all_books(os.path.join(directory, 'moliere'))
stats_moliere = compute_normalized_words_to_stats(all_paragraphs(moliere_dataset))
most_common_moliere = compute_most_common_normalized_words(stats_moliere, most_common_normalized_words_count)
moliere_total_word_count = sum([c[0] for c in stats_moliere.values()])
moliere_total_word_count_without_stopwords = sum([stats[0] for normalized_word,stats in stats_moliere.items() if normalized_word not in stop_words])



corneille_dataset = load_all_books(os.path.join(directory, 'corneille'))
stats_corneille = compute_normalized_words_to_stats(all_paragraphs(corneille_dataset))
most_common_corneille = compute_most_common_normalized_words(stats_corneille, most_common_normalized_words_count)
corneille_total_word_count = sum([c[0] for c in stats_corneille.values()])
corneille_total_word_count_without_stopwords = sum([stats[0] for normalized_word,stats in stats_corneille.items() if normalized_word not in stop_words])

normalized_words = list((set(stats_moliere.keys())|set(stats_corneille.keys())))
normalized_words.sort()
moliere_normalized_word_count_in_percentage = []
moliere_normalized_word_count = []
corneille_normalized_word_count_in_percentage = []
corneille_normalized_word_count = []
moliere_most_common_original_word = []
moliere_most_common_original_word_count = []
corneille_most_common_original_word = []
corneille_most_common_original_word_count = []


for normalized_word in normalized_words:
    if normalized_word in stats_moliere:
        stat = stats_moliere[normalized_word]
        moliere_normalized_word_count_in_percentage.append(stat[0]/moliere_total_word_count)
        moliere_normalized_word_count.append(stat[0])
        most_common_word, most_common_word_count = get_max_item(stat[1])
        moliere_most_common_original_word.append(most_common_word)
        moliere_most_common_original_word_count.append(most_common_word_count)
    else:
        moliere_normalized_word_count_in_percentage.append(0)
        moliere_normalized_word_count.append(0)
        moliere_most_common_original_word.append(None)
        moliere_most_common_original_word_count.append(None)
    if normalized_word in stats_corneille:
        stat = stats_corneille[normalized_word]
        corneille_normalized_word_count_in_percentage.append(stat[0]/corneille_total_word_count)
        corneille_normalized_word_count.append(stat[0])
        most_common_word, most_common_word_count = get_max_item(stat[1])
        corneille_most_common_original_word.append(most_common_word)
        corneille_most_common_original_word_count.append(most_common_word_count)
    else:
        corneille_normalized_word_count_in_percentage.append(0)
        corneille_normalized_word_count.append(0)
        corneille_most_common_original_word.append(None)
        corneille_most_common_original_word_count.append(None)

fhr_stats = pd.DataFrame(
    {'normalized_words': normalized_words,
    'moliere_count_in_percentage': moliere_normalized_word_count_in_percentage,
    'moliere_count': moliere_normalized_word_count,
    'corneille_count_in_percentage' : corneille_normalized_word_count_in_percentage,
    'corneille_count' : corneille_normalized_word_count,
    'moliere_most_common_original_word' : moliere_most_common_original_word,
    'moliere_most_common_original_word_count' : moliere_most_common_original_word_count,
    'corneille_most_common_original_word' : corneille_most_common_original_word,
    'corneille_most_common_original_word_count' : corneille_most_common_original_word_count,
    })

# on sauvegarde ces stats sur le disque
fhr_stats.to_csv(os.path.join(directory, 'stylometrie_stats.csv'), index=False, encoding='utf-8-sig')



<br><br><br>
<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 60px;"><b>1ère caractéristique</b></span>
<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 28px;">Nombre de 'mots signatures' d'un auteur présent dans un texte donné.</span><br>
<hr style="height:2px; border-width:0; color:black; background-color:black"><br>
<span style="font-size: 24px;"><i>Un 'mot signature' d'un auteur est un mot très fréquemment utilisé dans son œuvre.</i></span>
<br><br><br><br>

<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 48px;">1ère partie: </span><span style="font-size: 36px;">(basée sur la 1ère caractéristique)</span><br><br>
<span style="font-size: 36px;">Déterminer si un texte a été écrit par Molière.</span>
<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 24px;"><u>Méthode utilisée:</u></span><br>
<span style="font-size: 24px;">1. On calcule les 50 mots les plus courants chez Molière (ses 'mots signatures').</span><br>
<span style="font-size: 24px;">2. On calcule le nombre de mots signatures de Molière présent dans ce texte.</span><br>
<span style="font-size: 24px;">3. Si ce nombre dépasse un certain seuil, on attribue ce texte à Molière.</span>

## Affichage des 50 mots les plus fréquemment utilisés  chez Molière

### Nuage de mots-clés

In [ ]:
display_wordcloud(most_common_moliere, stats_moliere, "Molière")

### Graphique

In [ ]:
display_plot(most_common_moliere, stats_moliere, "Molière", 50, True)

# Affichage d'un graphique par nombre d'occurences
#display_plot(most_common_moliere, stats_moliere, "Molière", 50, False)

# Affichage des données brutes
#print("Mots les plus fréquents chez Molière\n", "\n".join([f'({normalized_word}, {frequency})'for normalized_word,frequency in most_common_moliere.items() if normalized_word not in stop_words ]))

## Exemple de calcul de cette 1ère caractéristique pour un texte de Molière

In [ ]:
texte_a_analyser = moliere_dataset['les_fourberies_de_scapin'][1]
print('Texte à analyser')
print('')
print('-'*50)
print(texte_a_analyser)
print('-'*50)
        
most_common_words_found_in_text = defaultdict(int)
for original_word in split_text(texte_a_analyser):
    normalized_word = compute_normalized_word(original_word)
    if normalized_word in most_common_moliere:
        most_common_words_found_in_text[normalized_word] += 1

title = f"{len(most_common_words_found_in_text)} mots différents de ce texte, apparaissant au total {sum(most_common_words_found_in_text.values())} fois,\nfont partie des {most_common_normalized_words_count} mots les plus courants chez Molière\n Valeur de la 1ère carcactéristiqe pour ce texte: {sum(most_common_words_found_in_text.values())}"
display_key_values_plot(list(most_common_words_found_in_text.keys()), list(most_common_words_found_in_text.values()), title, display_in_percentage=False)

## Affichage d'une courbe permettant de choisir la valeur optimale du seuil associé à la valeur de la 1ère caractéristique

In [ ]:
valeur_seuils = []
erreur_seuils = []
for seuil_1ere_caracteristique_moliere in range(0,40+1,2):
    (train_error, validation_error) = train_single_author(moliere_dataset, "Molière", corneille_dataset, seuil_1ere_caracteristique_moliere)
    valeur_seuils.append(seuil_1ere_caracteristique_moliere)
    erreur_seuils.append(train_error)
    print(f"seuil 1ère caractéristique={seuil_1ere_caracteristique_moliere} => Erreur(Molière)={round(100*train_error,1)}%")

# le code ci dessous (commenté) permet d'éviter de recalculer les valeurs
#print(valeur_seuils)
#print(erreur_seuils)
#valeur_seuils = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40]
#erreur_seuils = [0.5, 0.5, 0.4955555555555555, 0.4722222222222222, 0.42259259259259263, 0.35111111111111115, 0.2818518518518518, 0.2625925925925926, 0.26962962962962966, 0.3177777777777778, 0.3622222222222222, 0.39888888888888885, 0.43074074074074076, 0.4522222222222222, 0.4707407407407408, 0.4792592592592593, 0.4833333333333333, 0.48703703703703705, 0.48962962962962964, 0.49370370370370376, 0.49629629629629635]

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline


x_dense = np.linspace(min(valeur_seuils), max(valeur_seuils), 500)  # 500 points pour une courbe lisse
spline = make_interp_spline(valeur_seuils, erreur_seuils)
y_dense = spline(x_dense)

plt.figure(figsize=(20, 10))
plt.plot(x_dense, y_dense, label='Erreur', color='b')
plt.scatter(valeur_seuils, erreur_seuils, color='r')
plt.gca().tick_params(axis='y', which='major', labelsize=20) 
plt.gca().yaxis.set_major_formatter(PercentFormatter(1, decimals=0))
plt.xticks(fontsize=20)
plt.xlim(0, max(valeur_seuils))
plt.xlabel('Seuil associé à la 1ère caractéristique.\n(nombres minimum de mots signatures de Molière dans un texte pour le considérer comme écrit par Molière)', fontsize=20)
plt.ylabel('Erreur', fontsize=20)
plt.title("Evolution de l'erreur en fonction de la valeur du seuil associé à la 1ère caractéristique", fontsize=20)
plt.legend()


## On propose à l'étudiant de choisir le seuil associé à cette 1ère caractéristique

In [ ]:
# Seuil associé à cette 1ère caractéristique
# Si le nombre de mots signatures de Molière dans un texte est >= à ce seuil, alors ce texte sera attribué à Molière
# ce '10' sera modifé par l'étudiant
seuil_1ere_caracteristique = 10
(train_error, validation_error) = train_single_author(moliere_dataset, "Molière", corneille_dataset, seuil_1ere_caracteristique)
print(f"Erreur(Molière) avec 'seuil 1ère cacactéristique' = {seuil_1ere_caracteristique}  :  {round(100*validation_error,2)}%")

<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 48px;">2ème partie: </span><span style="font-size: 36px;">(basée sur la 1ère caractéristique)</span><br><br>
<span style="font-size: 36px;">Déterminer si un texte a été écrit par Corneille.</span>
<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 24px;"><u>Méthode utilisée:</u></span><br>
<span style="font-size: 24px;">1. On calcule les 50 mots les plus courants chez Corneille (ses 'mots signatures').</span><br>
<span style="font-size: 24px;">2. On calcule le nombre de mots signatures de Corneille présent dans ce texte.</span><br>
<span style="font-size: 24px;">3. Si ce nombre dépasse un certain seuil, on attribue ce texte à Corneille.</span>

## Affichage des 50 mots les plus fréquemment utilisés  chez Corneille

### Nuage de mots-clés

In [ ]:
display_wordcloud(most_common_corneille, stats_corneille, "Corneille")

### Graphique

In [ ]:
display_plot(most_common_corneille, stats_corneille, "Corneille", 50, True)

# Affichage d'un graphique par nombre d'occurence
#display_plot(most_common_corneille, stats_corneille, "Corneille", 50, False)

# Affichage des données brutes
#print("Mots les plus fréquents chez Corneille\n", "\n".join([f'({normalized_word}, {frequency})'for normalized_word,frequency in most_common_corneille.items() if normalized_word not in stop_words ]))

## Exemple de calcul de cette 1ère caractéristique pour un texte de Corneille

In [ ]:
texte_a_analyser = corneille_dataset['le_cid'][2]
print('Texte à analyser')
print('')
print('-'*50)
print(texte_a_analyser)
print('-'*50)

most_common_words_found_in_text = defaultdict(int)
for original_word in split_text(texte_a_analyser):
    normalized_word = compute_normalized_word(original_word)
    if normalized_word in most_common_corneille:
        most_common_words_found_in_text[normalized_word] += 1

title = f"{len(most_common_words_found_in_text)} mots différents de ce texte, apparaissant au total {sum(most_common_words_found_in_text.values())} fois,\nfont partie des {most_common_normalized_words_count} mots les plus courants chez Corneille\n Valeur de la 1ère carcactéristiqe pour ce texte: {sum(most_common_words_found_in_text.values())}"
display_key_values_plot(list(most_common_words_found_in_text.keys()), list(most_common_words_found_in_text.values()), title, display_in_percentage=False)

## Affichage d'une courbe permettant de choisir la valeur optimale du seuil associé à la valeur de la 1ère caractéristique

In [ ]:
valeur_seuils = []
erreur_seuils = []
for seuil_1ere_caracteristique_corneille in range(0,40+1,2):
    (train_error, validation_error) = train_single_author(corneille_dataset, "Corneille", moliere_dataset, seuil_1ere_caracteristique_corneille)
    valeur_seuils.append(seuil_1ere_caracteristique_corneille)
    erreur_seuils.append(validation_error)
    print(f"seuil 1ère caractéristique={seuil_1ere_caracteristique_corneille} => Erreur(Corneille)={round(100*validation_error,1)}%")

# le code ci dessous (commenté) permet d'éviter de recalculer les valeurs
#print(valeur_seuils)
#print(erreur_seuils)
#valeur_seuils = [0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40]
#erreur_seuils = [0.5, 0.49888888888888894, 0.4907407407407407, 0.4703703703703703, 0.4396296296296296, 0.39481481481481484, 0.3648148148148148, 0.35777777777777775, 0.36629629629629634, 0.40296296296296297, 0.43518518518518523, 0.4588888888888889, 0.4748148148148148, 0.4848148148148148, 0.49111111111111116, 0.4955555555555555, 0.49851851851851847, 0.4992592592592593, 0.5, 0.5, 0.5]

import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import make_interp_spline


x_dense = np.linspace(min(valeur_seuils), max(valeur_seuils), 500)  # 500 points pour une courbe lisse
spline = make_interp_spline(valeur_seuils, erreur_seuils)
y_dense = spline(x_dense)

plt.figure(figsize=(20, 10))
plt.plot(x_dense, y_dense, label='Erreur', color='b')
plt.scatter(valeur_seuils, erreur_seuils, color='r')
plt.gca().tick_params(axis='y', which='major', labelsize=20) 
plt.gca().yaxis.set_major_formatter(PercentFormatter(1, decimals=0))
plt.xticks(fontsize=20)
plt.xlim(0, max(valeur_seuils))
plt.xlabel('Seuil associé à la 1ère caractéristique.\n(nombres minimum de mots signatures de Corneille dans un texte pour le considérer comme écrit par Corneille)', fontsize=20)
plt.ylabel('Erreur', fontsize=20)
plt.title("Evolution de l'erreur en fonction de la valeur du seuil associé à la 1ère caractéristique", fontsize=20)
plt.legend()


## On propose à l'étudiant de choisir le seuil associé à cette 1ère caractéristique

In [ ]:
# Seuil associé à cette 1ère caractéristique
# Si le nombre de mots signatures de Molière dans un texte est >= à ce seuil, alors ce texte sera attribué à Molière
# ce '10' sera modifé par l'étudiant
seuil_1ere_caracteristique = 10
(train_error, validation_error) = train_single_author(corneille_dataset, "Corneille", moliere_dataset, seuil_1ere_caracteristique)
print(f"Erreur(Corneille) avec 'seuil 1ère cacactéristique' = {seuil_1ere_caracteristique}  :  {round(100*validation_error,2)}%")

<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 48px;">3ème partie: </span><span style="font-size: 36px;">(basée sur la 1ère caractéristique)</span><br><br>
<span style="font-size: 36px;">Déterminer si un texte a été écrit par Molière ou par Corneille.</span>
<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 24px;"><u>Méthode utilisée:</u></span><br>
<span style="font-size: 24px;">1. On calcule les 50 mots les plus courants chez Molière (ses 'mots signatures').</span><br>
<span style="font-size: 24px;">2. On calcule les 50 mots les plus courants chez Corneille (ses 'mots signatures').</span><br>
<span style="font-size: 24px;">3. On calcule le nombre de mots signatures de Molière présent dans ce texte.</span><br>
<span style="font-size: 24px;">4. On calcule le nombre de mots signatures de Corneille présent dans ce texte.</span><br>
<span style="font-size: 24px;">5. On attribue le texte à l'auteur ayant le plus de mots signatures dans ce texte.</span>

## PRIVE: Entraînement et calcul des métriques avec deux auteurs

In [ ]:

def compute_confusion_matrix_all_authors(text_moliere: List[str], text_corneille: List[str], most_common_words_moliere: dict , most_common_words_corneille: dict, verbose:bool = False) ->Tuple[int,int,int,int]:
    TP = 0 # y_true = Molière ,  y_pred = Molière
    TN = 0 # y_true = Corneille, y_pred = Corneille
    FN = 0 # y_true = Molière,   y_pred = Corneille
    FP = 0 # y_true = Corneille, y_pred = Molière 
    for t in text_moliere:
        score_moliere = calcul_1ere_caracteristique(t, most_common_words_moliere)
        score_corneille = calcul_1ere_caracteristique(t, most_common_words_corneille)
        if score_moliere>score_corneille:
            if TP == 0 and verbose:
                print(f'\nExemple de TP (Texte de Molière, bien identifié, score Molière: {round(score_moliere,4)}, score Corneille: {round(score_corneille,4)}):\n{t}\n')
            TP += 1
        else:
            if FN == 0 and verbose:
                print(f'\nExemple de FN (Texte de Molière, mal identifié, score Molière: {round(score_moliere,4)}, score Corneille: {round(score_corneille,4)}):\n{t}\n')
            FN += 1
    for t in text_corneille:
        score_moliere = calcul_1ere_caracteristique(t, most_common_words_moliere)
        score_corneille = calcul_1ere_caracteristique(t, most_common_words_corneille)
        if score_moliere>score_corneille:
            if FP == 0 and verbose:
                print(f'\nExemple de FP (Texte de Corneille, mal identifié, score Molière: {round(score_moliere,4)}, score Corneille: {round(score_corneille,4)}):\n{t}\n')
            FP += 1
        else:
            if TN == 0 and verbose:
                print(f'\nExemple de TN (Texte de Corneille, bien identifié, score Molière: {round(score_moliere,4)}, score Corneille: {round(score_corneille,4)}):\n{t}\n')
            TN += 1
    return (TP,TN,FP,FN)
        
    
def prediction_moliere_vs_corneille_1ere_caracteristique(most_common_count, verbose: bool = False) -> Tuple[float,float]:
    random.seed(42)
    if verbose: 
        print(f'\nMoliere Dataset: {paragraph_count(moliere_dataset)} paragraphes ({word_count(moliere_dataset)} mots) venant de {len(moliere_dataset)} oeuvres:\n{list(moliere_dataset.keys())}')
        print(f'\nCorneille Dataset: {paragraph_count(corneille_dataset)} paragraphes ({word_count(corneille_dataset)} mots) venant de {len(corneille_dataset)} oeuvres:\n{list(corneille_dataset.keys())}')

    train_moliere,validation_moliere,train_corneille,validation_corneille = split_train_validation_all_authors(moliere_dataset, corneille_dataset, percentage_in_train)

    if verbose: 
        print(f'\nMoliere Train Dataset: {paragraph_count(train_moliere)} paragraphes ({word_count(train_moliere)} mots) venant de {len(train_moliere)} oeuvres:\n{list(train_moliere.keys())}')
        print(f'\nMoliere Validation Dataset: {paragraph_count(validation_moliere)} paragraphes ({word_count(validation_moliere)} mots) venant de {len(validation_moliere)} oeuvres:\n{list(validation_moliere.keys())}')
        print(f'\nCorneille Train Dataset: {paragraph_count(train_corneille)} paragraphes ({word_count(train_corneille)} mots) venant de {len(train_corneille)} oeuvres:\n{list(train_corneille.keys())}')
        print(f'\nCorneille Validation Dataset: {paragraph_count(validation_corneille)} paragraphes ({word_count(validation_corneille)} mots) venant de {len(validation_corneille)} oeuvres:\n{list(validation_corneille.keys())}')

    train_normalized_words_to_stats_moliere = compute_normalized_words_to_stats(all_paragraphs(train_moliere))
    train_normalized_words_to_stats_corneille = compute_normalized_words_to_stats(all_paragraphs(train_corneille))

    # we only keep the most common words
    train_most_common_moliere = compute_most_common_normalized_words(train_normalized_words_to_stats_moliere, most_common_count)
    train_most_common_corneille = compute_most_common_normalized_words(train_normalized_words_to_stats_corneille, most_common_count)
    train_error = compute_error(*compute_confusion_matrix_all_authors(all_paragraphs(train_moliere), all_paragraphs(train_corneille), train_most_common_moliere , train_most_common_corneille, verbose))
    validation_error = compute_error(*compute_confusion_matrix_all_authors(all_paragraphs(validation_moliere), all_paragraphs(validation_corneille), train_most_common_moliere , train_most_common_corneille, verbose))
    return train_error, validation_error



## Résultats de cette méthode (basée sur la 1ère caractéristique) en ayant choisit les 50 mots les plus courants de chaque auteur comme leurs mots signature

In [ ]:
(train_error,validation_error) = prediction_moliere_vs_corneille_1ere_caracteristique(most_common_normalized_words_count)
print(f'Erreur(Molière ou Corneille?)= {round(100*validation_error,1)}%')


## Pour améliorer ces résultats, on peut proposer à l'étudiant de modifier le nombre de mots signatures associés à chaque auteur (ci dessus: 50)

### On affiche à l'étudiant la valeur de l'erreur pour différentes valeurs du nombre de mots signatures

In [ ]:
most_common_counts = [50,1000]+list(range(2000,16000+1,2000))
error_for_most_common_counts = []
for most_common_count in most_common_counts:
    (train_error,validation_error) = prediction_moliere_vs_corneille_1ere_caracteristique(most_common_count)
    print(f'Erreur(Molière ou Corneille?) if most_common_count={most_common_count} = {round(100*validation_error,1)}%')
    error_for_most_common_counts.append(validation_error)

# le code ci dessous (commenté) permet d'éviter de recalculer les valeurs
#print(most_common_counts)
#print(error_for_most_common_counts)
#most_common_counts = [50, 1000, 2000, 4000, 6000, 8000, 10000, 12000, 14000, 16000, 18000]
#error_for_most_common_counts = [0.0738255033557047, 0.030201342281879207, 0.01342281879194629, 0.010067114093959773, 0.02684563758389258, 0.023489932885906062, 0.043624161073825496, 0.05033557046979864, 0.06375838926174493, 0.06375838926174493, 0.06375838926174493]

x_dense = np.linspace(min(most_common_counts), max(most_common_counts), 500)  # 500 points pour une courbe lisse
spline = make_interp_spline(most_common_counts, error_for_most_common_counts)
y_dense = spline(x_dense)

plt.figure(figsize=(20, 10))
plt.plot(x_dense, y_dense, label='Erreur', color='b')
plt.scatter(most_common_counts, error_for_most_common_counts, color='r')
plt.gca().tick_params(axis='y', which='major', labelsize=20) 
plt.gca().yaxis.set_major_formatter(PercentFormatter(1, decimals=0))
plt.xticks(fontsize=20)
plt.xlim(0, max(most_common_counts))
plt.xlabel('Nombres de mots signatures chez chaque auteur', fontsize=20)
plt.ylabel('Erreur pour distinguer des oeuvres de Molière et Corneille', fontsize=20)
plt.title("Evolution de l'erreur en fonction du nombre de mots signatures chez chaque auteur", fontsize=20)
plt.legend()
            
    

## On propose à l'étudiant de choisir la valeur de la caractéristique

In [ ]:
#la caractéristique à améliorer
# ce '50' sera modifé par l'étudiant
nombre_de_mots_signatures_chez_chaque_auteur = 50

(train_error,validation_error) = prediction_moliere_vs_corneille_1ere_caracteristique(nombre_de_mots_signatures_chez_chaque_auteur)
print()
print('-'*80)
print(f'Erreur(Molière ou Corneille?)={round(100*validation_error,1)}%')
print('-'*80)

<br><br><br>
<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 60px;"><b>2ème caractéristique</b></span>
<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 28px;">Le nombre de mots (dans un texte donné) nettement plus fréquents chez un auteur par rapport à un autre.</span><br>
<hr style="height:2px; border-width:0; color:black; background-color:black"><br>
<br><br><br><br>

# PRIVE: outil de recherche de textes très spécifiques à Molière ou Corneille

In [ ]:
def compute_word_score_moliere_vs_corneille(count_moliere:int, total_words_moliere:int, count_corneille:int, total_words_corneille:int ):
    if count_moliere+count_corneille<30:
        return 0
    if count_moliere == 0:
        return -1
    if count_corneille == 0:
        return 1
    percentage_moliere = count_moliere/total_words_moliere
    percentage_corneille = count_corneille/total_words_corneille
    if percentage_moliere>2*percentage_corneille:
        return 1
    if percentage_corneille>2*percentage_moliere:
        return -1
    return 0

def find_most_distinctive_lines(is_moliere: bool) -> None:
    min_score_moliere = 0
    max_score_moliere = 0
    
    author_dataset = moliere_dataset if is_moliere else corneille_dataset
    
    for book_name, paragraphs in author_dataset.items():
        for paragraph in paragraphs:
            for line in paragraph.splitlines():
                words = split_text(line)
                if len(words)<5:
                    continue
                line_score_moliere_vs_corneille = 0
                comment_moliere = ""
                comment_corneille = ""
                for original_word in words:
                    normalized_word = compute_normalized_word(original_word)
                    count_moliere = stats_moliere[normalized_word][0] if normalized_word in stats_moliere else 0
                    count_corneille = stats_corneille[normalized_word][0] if normalized_word in stats_corneille else 0
                    word_score_moliere_vs_corneille = compute_word_score_moliere_vs_corneille(count_moliere, moliere_total_word_count, count_corneille, corneille_total_word_count)
                    if word_score_moliere_vs_corneille == 0:
                        continue
                    if is_moliere:
                        comment = f"{original_word} ({count_moliere} vs {count_corneille}) "
                    else:
                        comment = f"{original_word} ({count_corneille} vs {count_moliere}) "
                    if word_score_moliere_vs_corneille>0:
                        comment_moliere += comment
                    else:
                        comment_corneille += comment
                    line_score_moliere_vs_corneille += word_score_moliere_vs_corneille
                if is_moliere and comment_corneille:
                    continue
                if not is_moliere and comment_moliere:
                    continue
                if abs(line_score_moliere_vs_corneille)>=5 and (len(comment_moliere)==0 or len(comment_corneille)==0):
                #if total_score_moliere<min_score_moliere or line_score_moliere_vs_corneille>max_score_moliere:
                    min_score_moliere = min(min_score_moliere,line_score_moliere_vs_corneille)
                    max_score_moliere = max(max_score_moliere,line_score_moliere_vs_corneille)
                    print('-'*50)
                    print(f"Oeuvre de {'Molière' if is_moliere else 'Corneille'}: {book_name}")
                    print(line)
                    print(f'Score: {line_score_moliere_vs_corneille}')
                    if comment_moliere:
                        print(f'avantage Molière: {comment_moliere}')
                    if comment_corneille:
                        print(f'avantage Corneille: {comment_corneille}')
                    print('-'*50)
                    print()

'''
Exemples de textes trouvés par cet outil:

--------------------------------------------------
Oeuvre de Molière: le_malade_imaginaire
Qu'il se fasse médecin, je consens au mariage. Oui, faites-vous médecin, je vous donne ma fille.
Score: 5
avantage Molière: médecin (208 vs 1) mariage (181 vs 24) Oui (849 vs 165) médecin (208 vs 1) fille (415 vs 128) 

--------------------------------------------------
Oeuvre de Corneille: polyeucte
Ton courage était bon, ton devoir l'a trahi.
Score: -7
avantage Corneille: Ton (759 vs 224) courage (206 vs 44) était (86 vs 1) ton (759 vs 224) devoir (182 vs 69) l'a (121 vs 72) trahi (22 vs 8) 
'''                    
                    
                
print('-'*50)
print('Recherche de lignes spécifiques à Molière')
find_most_distinctive_lines(True)
print()
print('-'*50)
print('Recherche de lignes spécifiques à Corneille')
find_most_distinctive_lines(False)
print()
    


# Mise en situation:
## On veut identifier les auteurs des 2 phrases suivantes:
### - Oui, faites-vous médecin, je vous donne ma fille.
### - Ton courage était bon, ton devoir l'a trahi.

## On montre les fréquences d'apparition de certains mots chez Molière et Corneille 

In [ ]:
mots = ['courage', 'devoir', 'fille', 'médecin', 'oui', 'trahi']

'''
for word in mots:
    normalized_word = compute_normalized_word(word)
    print(f"Le mot '{word}':")
    print(f'\test présent {stats_moliere[normalized_word][0]} fois chez Molière:  ', stats_moliere[normalized_word][1])
    print(f'\test présent {stats_corneille[normalized_word][0]} fois chez Corneille:', stats_corneille[normalized_word][1])
'''
df = create_table_with_occurences_corneille_moliere(mots)

def format_percentage(value):
    return f'{round(100*value,3)}%'

df['Fréquence chez Molière'] = df['Fréquence Molière'].apply(format_percentage)
df['Fréquence chez Corneille'] = df['Fréquence Corneille'].apply(format_percentage)
df[['Fréquence chez Molière', 'Fréquence chez Corneille']]



## En se basant sur le tableau d'occurences ci dessus, qui de Molière ou Corneille a probalement écrit cette ligne:
### "Oui, faites-vous médecin, je vous donne ma fille."

In [ ]:
# remplacer le "XXX" ci dessous par "Molière" ou par "Corneille"
auteur = "XXX"

## En se basant sur le tableau d'occurences ci dessus, qui de Molière ou Corneille a probalement écrit cette ligne:
### "Ton courage était bon, ton devoir l'a trahi."

In [ ]:
# remplacer le "XXX" ci dessous par "Molière" ou par "Corneille"
auteur = "XXX"

<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 48px;">4ème partie: </span><span style="font-size: 36px;">(basée sur la 2ème caractéristique)</span><br><br>
<span style="font-size: 36px;">Déterminer si un texte a été écrit par Molière ou Corneille.</span>
<hr style="height:2px; border-width:0; color:black; background-color:black">
<span style="font-size: 24px;"><u>Méthode utilisée:</u></span><br>
<span style="font-size: 24px;">1. Pour chaque mot du texte, on calcule sa fréquence d'apparition chez Molière et chez Corneille.</span><br>
<span style="font-size: 24px;">2. On calcule le nombre de mots de ce texte 2 fois plus fréquents chez Molière que chez Corneille.</span><br>
<span style="font-size: 24px;">3. On calcule le nombre de mots de ce texte 2 fois plus fréquents chez Corneille que chez Molière.</span><br>
<span style="font-size: 24px;">4. On attribue le texte à l'auteur ayant le plus de mots plus fréquents chez lui.</span>

In [ ]:
def calcul_valeur_2eme_caracteristique_moliere_et_corneille(text:str, train_stats_moliere, train_stats_corneille, moliere_word_count, corneille_word_count, multiplier: float) -> Tuple[int,int]:
    valeur_2eme_caracteristique_moliere = 0
    valeur_2eme_caracteristique_corneille = 0
    for original_word in split_text(text):
        normalized_word = compute_normalized_word(original_word)
        if normalized_word in stop_words:
            continue
        frequency_moliere = 0
        if normalized_word in train_stats_moliere:
            frequency_moliere = train_stats_moliere[normalized_word][0]/moliere_total_word_count_without_stopwords
        frequency_corneille = 0
        if normalized_word in train_stats_corneille:
            frequency_corneille = train_stats_corneille[normalized_word][0]/corneille_total_word_count_without_stopwords
        if frequency_moliere>(multiplier*frequency_corneille):
            valeur_2eme_caracteristique_moliere += 1
        if frequency_corneille>(multiplier*frequency_moliere):
            valeur_2eme_caracteristique_corneille += 1
    return valeur_2eme_caracteristique_moliere,valeur_2eme_caracteristique_corneille

def compute_confusion_matrix_all_authors_frequency(text_moliere: List[str], text_corneille: List[str], train_stats_moliere, train_stats_corneille, multiplier: float) ->Tuple[int,int,int,int]:
    TP = 0 # y_true = Molière ,  y_pred = Molière
    TN = 0 # y_true = Corneille, y_pred = Corneille
    FN = 0 # y_true = Molière,   y_pred = Corneille
    FP = 0 # y_true = Corneille, y_pred = Molière 
    
    moliere_word_count = sum([stats[0] for normalized_word,stats in train_stats_moliere.items() if normalized_word not in stop_words])
    corneille_word_count = sum([stats[0] for normalized_word,stats in train_stats_corneille.items() if normalized_word not in stop_words])
    for t in text_moliere:
        (valeur_2eme_caracteristique_moliere,valeur_2eme_caracteristique_corneille) = calcul_valeur_2eme_caracteristique_moliere_et_corneille(t, train_stats_moliere, train_stats_corneille, moliere_word_count, corneille_word_count, multiplier)
        if valeur_2eme_caracteristique_moliere>valeur_2eme_caracteristique_corneille:
            TP += 1
        else:
            FN += 1
    for t in text_corneille:
        (valeur_2eme_caracteristique_moliere,valeur_2eme_caracteristique_corneille) = calcul_valeur_2eme_caracteristique_moliere_et_corneille(t, train_stats_moliere, train_stats_corneille, moliere_word_count, corneille_word_count, multiplier)
        if valeur_2eme_caracteristique_moliere>valeur_2eme_caracteristique_corneille:
            FP += 1
        else:
            TN += 1
    return (TP,TN,FP,FN)
        
    
def prediction_moliere_vs_corneille_2eme_caracteristique(multiplier: float) -> Tuple[float,float]:
    random.seed(42)
    train_moliere,validation_moliere,train_corneille,validation_corneille = split_train_validation_all_authors(moliere_dataset, corneille_dataset, percentage_in_train)
    train_stats_moliere = compute_normalized_words_to_stats(all_paragraphs(train_moliere))
    train_stats_corneille = compute_normalized_words_to_stats(all_paragraphs(train_corneille))
    train_error = compute_error(*compute_confusion_matrix_all_authors_frequency(all_paragraphs(train_moliere), all_paragraphs(train_corneille), train_stats_moliere, train_stats_corneille, multiplier))
    validation_error = compute_error(*compute_confusion_matrix_all_authors_frequency(all_paragraphs(validation_moliere), all_paragraphs(validation_corneille), train_stats_moliere, train_stats_corneille, multiplier))
    return train_error, validation_error


## Affichage d'une courbe permettant de choisir la valeur optimale du coéfficient 'K' à partir duquel on considére un mot beaucoup plus courant chez l'un des auteurs

In [ ]:
multipliers = list(range(1, 7+1))

error_for_multipliers = []
for multiplier in multipliers:
    (train_error,validation_error) = prediction_moliere_vs_corneille_2eme_caracteristique(multiplier)
    print(f'Erreur(Molière ou Corneille?) if multiplier={multiplier} = {round(100*validation_error,1)}%')
    error_for_multipliers.append(validation_error)


x_dense = np.linspace(min(multipliers), max(multipliers), 500)  # 500 points pour une courbe lisse
spline = make_interp_spline(multipliers, error_for_multipliers)
y_dense = spline(x_dense)

plt.figure(figsize=(20, 10))
plt.plot(x_dense, y_dense, label='Erreur', color='b')
plt.scatter(multipliers, error_for_multipliers, color='r')
plt.gca().tick_params(axis='y', which='major', labelsize=20) 
plt.gca().yaxis.set_major_formatter(PercentFormatter(1, decimals=0))
plt.xticks(fontsize=20)
plt.xlim(min(multipliers), max(multipliers))
plt.xlabel('Coéfficient K (à partir duquel on considére un mot nettement plus fréquent chez auteur)', fontsize=20)
plt.ylabel('Erreur pour distinguer des oeuvres de Molière et Corneille', fontsize=20)
plt.title("Evolution de l'erreur en fonction de ce coéfficient K", fontsize=20)
plt.legend()
            
    

## On propose à l'étudiant de choisir la valeur de la caractéristique

In [ ]:
#la caractéristique à améliorer
# ce '2' sera modifé par l'étudiant
multiplier = 2
(train_error,validation_error) = prediction_moliere_vs_corneille_2eme_caracteristique(multiplier)
print(f'Erreur(Molière ou Corneille?)= {round(100*validation_error,1)}%')


## PRIVE: Erreur pour chaque oeuvre utilisée

In [ ]:
optimal_most_common_normalized_words_count = 3500
most_common_moliere = compute_most_common_normalized_words(stats_moliere, optimal_most_common_normalized_words_count)
most_common_corneille = compute_most_common_normalized_words(stats_corneille, optimal_most_common_normalized_words_count)

print('-'*80+'\nErreur pour chaque oeuvre de Moliere\n'+'-'*80)
error_moliere = dict()
for book_path in all_txt_files_in_directory(os.path.join(directory, 'moliere')):
    (TP,TN,FP,FN) = compute_confusion_matrix_all_authors(split_book_into_paragraphs(book_path), [], most_common_moliere , most_common_corneille, 0)
    #print(f"Error '{pathlib.Path(book_path).stem}': {round(compute_error(TP,TN,FP,FN),4)}")
    error_moliere[pathlib.Path(book_path).stem] = compute_error(TP,TN,FP,FN)
for e in sorted(error_moliere.items(), key=lambda x: x[1]):
    print(e)

print()
print('-'*80+'\nErreur pour chaque oeuvre de Corneille\n'+'-'*80)
erreur_corneille = dict()
for book_path in all_txt_files_in_directory(os.path.join(directory, 'corneille')):
    (TP,TN,FP,FN) = compute_confusion_matrix_all_authors([], split_book_into_paragraphs(book_path), most_common_moliere , most_common_corneille, 0)
    erreur_corneille[pathlib.Path(book_path).stem] = compute_error(TP,TN,FP,FN)
for e in sorted(erreur_corneille.items(), key=lambda x: x[1]):
    print(e)
    
    